In [2]:
import re
from pathlib import Path
import glob

62 PXD-files where deemed 'useable' from the ionbot excel spreadsheet. <br>
Now, it will be checked whether they are already processed in ionbot.

In [3]:
found_pxd = ['PXD004051', 'PXD006421', 'PXD005620', 'PXD002676', 'PXD001952', 'PXD007885', 'PXD007906', 'PXD021281', 'PXD006653', 'PXD002842', 'PXD003594', 'PXD006035', 'PXD008719', 'PXD001668', 'PXD009185', 'PXD005507', 'PXD001247', 'PXD007837', 'PXD008035', 'PXD001974', 'PXD002057', 'PXD008381', 'PXD010012', 'PXD005045', 'PXD009933', 'PXD002032', 'PXD010538', 'PXD006614', 'PXD009600', 
'PXD009442', 'PXD010306', 'PXD009557', 'PXD009560', 'PXD003252', 'PXD003587', 'PXD007543', 'PXD008693', 'PXD003362', 'PXD011580', 'PXD010256', 'PXD000426', 'PXD009861', 'PXD006419', 'PXD008967', 'PXD000612', 'PXD001592', 'PXD003438', 'PXD004824', 'PXD014300', 'PXD018450', 'PXD018625', 'PXD017391', 'PXD018918', 'PXD011961', 'PXD013541', 'PXD018066', 'PXD016742', 'PXD014777', 'PXD017898', 'PXD014381', 'PXD016119', 'PXD015571', 'PXD009149']

In [4]:
root = Path("/home/compomics/mounts")

pxd_dir = glob.glob("/home/compomics/mounts/*/*/PRIDE_DATA/*")
print(pxd_dir[:5])
len(pxd_dir)

['/home/compomics/mounts/conode53/pride2/PRIDE_DATA/PXD001857', '/home/compomics/mounts/conode53/pride2/PRIDE_DATA/PXD000272', '/home/compomics/mounts/conode53/pride2/PRIDE_DATA/PXD004019', '/home/compomics/mounts/conode53/pride2/PRIDE_DATA/PXD004113', '/home/compomics/mounts/conode53/pride2/PRIDE_DATA/PXD004745']


2272

In [5]:
PXD_reprocessed = []
dir_PXD_reprocessed = []
regex = re.compile(r"(PXD)(\d+)")
for i in pxd_dir:
    result = regex.findall(i)
    if result != []:    
        #PXD_reprocessed.append("".join(result[0]))
        if "".join(result[0]) in found_pxd:
            dir_PXD_reprocessed.append(i)
            PXD_reprocessed.append("".join(result[0]))



In [6]:
print(len(PXD_reprocessed), PXD_reprocessed[:5])
print(len(dir_PXD_reprocessed), dir_PXD_reprocessed[:5])

49 ['PXD001592', 'PXD006421', 'PXD007906', 'PXD002842', 'PXD003594']
49 ['/home/compomics/mounts/conode53/pride2/PRIDE_DATA/PXD001592', '/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD006421', '/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD007906', '/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD002842', '/home/compomics/mounts/conode53/pride/PRIDE_DATA/PXD003594']


It seems like 49 of the 63 PXDs are reprocessed.

In [10]:
import openpyxl, os
check1 = {"PXD": 0, "RAW": 0}
wb = openpyxl.Workbook()
sheet = wb["Sheet"]
sheet["A1"], sheet["B1"], sheet["C1"] = "PXD", "RAW", "annotation"

for i in dir_PXD_reprocessed:
    check1['PXD'] += 1
    sheet.cell(row = sheet.max_row + 2, column = 1).value = os.path.basename(i)
    for count, x in enumerate(glob.glob(f"{i}/RAW/*")):
        check1['RAW'] += 1
        if count == 0:
            sheet.cell(row = sheet.max_row, column = 2). value = os.path.basename(x)
            continue
        sheet.cell(row = sheet.max_row + 1, column = 2). value = os.path.basename(x)

wb.save("annotation_excel.xlsx")

check1

{'PXD': 49, 'RAW': 1893}